<a href="https://colab.research.google.com/github/Zarak00/Projects/blob/main/Satellite_Reconciliation/Satellite_Reconciliation_Partial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import os
import pandas as pd
from datetime import datetime

In [28]:
folder_path = "/content/Data"  # Update with the correct folder path
os.chdir(folder_path)


In [29]:
merged_data = pd.DataFrame()
for file_name in os.listdir():
    if file_name.endswith('.txt'):
        file_path = os.path.join(folder_path, file_name)
        data = pd.read_csv(file_path, delimiter=',')  # Assuming tab-separated data
        merged_data = merged_data.append(data)


<ipython-input-29-8c593c7869c8>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_data = merged_data.append(data)
<ipython-input-29-8c593c7869c8>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_data = merged_data.append(data)
<ipython-input-29-8c593c7869c8>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_data = merged_data.append(data)
<ipython-input-29-8c593c7869c8>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_data = merged_data.append(data)
<ipython-input-29-8c593c7869c8>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste

In [30]:
merged_data.to_csv('/content/Data/merged_data.csv', index=False)


In [31]:
df1 = pd.read_csv('/content/Data/merged_data.csv')

In [32]:
df1['Usage'] = df1['Usage'].astype('Int64')

In [33]:
df1['Usage Unit'] = pd.to_numeric(df1['Usage Unit'], errors='coerce').fillna(0).astype(int)


In [34]:
df1['Aircraft'] = df1['Aircraft'].str.replace('-', '')

In [35]:
df1['Act Period'] = pd.to_datetime(df1['Act Period']).dt.strftime('%m-%d-%Y')
df1 = df1.sort_values('Act Period')

In [36]:
df1.head(1)

,Act Period,Charge Type,Aircraft,Medium Type,Protocol Used,Protocol Billed,Tfc Type,Tfc Direction,Medium / Zone,Station / Site,Avg Unit Rate,Usage,Usage Unit,Tfc Charge
0,01-01-2021,Aircraft,0001052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.995,<NA>,0,18.995


In [37]:
# Replace 'your_created_file.csv' with the actual path and filename of your CSV file
df2 = pd.read_csv('/content/Data/2022 SITA Credit FJ analysis.csv')

In [38]:
df2 = df2.iloc[:, [0]]

In [39]:
df2.tail(5)

,AIRCRAFT
266,VQBRZ
267,VTAGL
268,XAGIE
269,XAHNF
270,YLABL


#Performing Groupby

In [40]:
filtered_df_1 = df1[df1['Charge Type'] == 'Traffic']

In [41]:
usage_df = filtered_df_1.groupby('Aircraft')['Usage'].sum().reset_index()
usage_df = usage_df.rename(columns={'Aircraft': 'AIRCRAFT', 'Usage': 'Usage'})
usage_df

,AIRCRAFT,Usage
0,0001052,160
1,0001053,265
2,0015441,1
3,010015,1759
4,0100301,40
...,...,...
4461,ZSMIG,1689
4462,ZSNEX,3
4463,ZZ664,99
4464,ZZ665,1599


In [42]:
filtered_df_2 = df1[df1['Charge Type'] == 'Aircraft']

In [43]:
avg_unit_rate_df = filtered_df_2.groupby('Aircraft')['Avg Unit Rate'].count().reset_index()
avg_unit_rate_df = avg_unit_rate_df.rename(columns={'Aircraft': 'AIRCRAFT', 'Avg Unit Rate': 'Avg Unit Rate'})
avg_unit_rate_df

,AIRCRAFT,Avg Unit Rate
0,0001052,5
1,0001053,6
2,0015441,1
3,010015,9
4,0100301,4
...,...,...
4496,ZSMIG,12
4497,ZSNEX,2
4498,ZZ664,10
4499,ZZ665,8


In [44]:
# Merge the two DataFrames based on the 'AIRCRAFT' column
merged_df = df2.merge(usage_df, left_on='AIRCRAFT', right_on='AIRCRAFT', how='left')

# Calculate the new column based on the conditions
merged_df['2018 Usage'] = merged_df['Usage'].fillna(0) * 0.1

# Replace any remaining NaN values with 0
merged_df['2018 Usage'].fillna(0, inplace=True)


In [45]:
merged_df

,AIRCRAFT,Usage,2018 Usage
0,0,<NA>,0.0
1,3526,<NA>,0.0
2,95009,782,78.2
3,295513,<NA>,0.0
4,940315,<NA>,0.0
...,...,...,...
266,VQBRZ,<NA>,0.0
267,VTAGL,<NA>,0.0
268,XAGIE,1814,181.4
269,XAHNF,413,41.3


In [46]:
# Merge the two DataFrames based on the 'AIRCRAFT' column
merged_df_2 = df2.merge(avg_unit_rate_df, left_on='AIRCRAFT', right_on='AIRCRAFT', how='left')

# Calculate the new column based on the conditions
merged_df_2['2018 Subscription'] = merged_df_2['Avg Unit Rate'].fillna(0) * 60

# Replace any remaining NaN values with 0
merged_df_2['2018 Subscription'].fillna(0, inplace=True)



In [47]:
merged_df_2

,AIRCRAFT,Avg Unit Rate,2018 Subscription
0,0,NaN,0.0
1,3526,NaN,0.0
2,95009,8.0,480.0
3,295513,NaN,0.0
4,940315,NaN,0.0
...,...,...,...
266,VQBRZ,NaN,0.0
267,VTAGL,NaN,0.0
268,XAGIE,12.0,720.0
269,XAHNF,9.0,540.0


In [48]:
# Perform inner join on "AIRCRAFT" column
df3 = pd.merge(merged_df, merged_df_2, on="AIRCRAFT")

# Drop the "Usage" and "Avg Unit Rate" columns from the dataframe
df3 = df3.drop(["Usage", "Avg Unit Rate"], axis=1)

# Create the new "credit" column
df3['credit'] = df3['2018 Usage'] + df3['2018 Subscription']


df3

,AIRCRAFT,2018 Usage,2018 Subscription,credit
0,0,0.0,0.0,0.0
1,3526,0.0,0.0,0.0
2,95009,78.2,480.0,558.2
3,295513,0.0,0.0,0.0
4,940315,0.0,0.0,0.0
...,...,...,...,...
266,VQBRZ,0.0,0.0,0.0
267,VTAGL,0.0,0.0,0.0
268,XAGIE,181.4,720.0,901.4
269,XAHNF,41.3,540.0,581.3


In [49]:
# Calculate the sum of all columns except "AIRPORT"
last_row = df3.drop("AIRCRAFT", axis=1).sum()

# Create a new row using the calculated sum
final_df = df3.append(last_row, ignore_index=True)

<ipython-input-49-d7b2feef7dfb>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = df3.append(last_row, ignore_index=True)


##Change Column Names
(only if you want)

In [50]:
final_df = final_df.rename(columns={'2018 Usage': '2021 Usage', '2018 Subscription': '2021 Subscription'})

In [51]:
final_df

,AIRCRAFT,2021 Usage,2018 Subscription,credit
0,0,0.0,0.0,0.0
1,3526,0.0,0.0,0.0
2,95009,78.2,480.0,558.2
3,295513,0.0,0.0,0.0
4,940315,0.0,0.0,0.0
...,...,...,...,...
267,VTAGL,0.0,0.0,0.0
268,XAGIE,181.4,720.0,901.4
269,XAHNF,41.3,540.0,581.3
270,YLABL,0.0,0.0,0.0


##Download The File
- Change the file name if you feel like

In [52]:
final_df.to_csv('2021_report_1.csv', index=False)
